In [1]:
import pandas as pd
import numpy as np
import json
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas datafram
!pip install bs4
from bs4 import BeautifulSoup

     |████████████████████████████████| 102kB 4.5MB/s ta 0:00:01
  Created wheel for bs4: filename=bs4-0.0.1-cp36-none-any.whl size=1273 sha256=d9c538659bc0c1848799dc849b49288bd96f4dc73b5ff5d3de14993e1e86a6af
  Stored in directory: /home/jovyan/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M';
response = requests.get(url);
soup = BeautifulSoup(response.text,'html.parser')

In [3]:
table = soup.find("table", {"class":"wikitable sortable"}).tbody

Finding rows and columns from html, deleting unnecessary text ("\n")

In [4]:
rows = table.find_all('tr')
cols = [v.text.replace('\n','') for v in rows[0].find_all('th')]

In [5]:
df = pd.DataFrame(columns=cols)
for i in range(1,len(rows)):
    tds = [v.text.replace("\n"," ") for v in rows[i].find_all('td')]
    df.loc[i] = tds
df.drop(df[df['Borough'] == 'Not assigned'].index,inplace=True);
df = df.groupby(by=['Postcode','Borough'],sort= False).agg(lambda x: ', '.join(x));
df.reset_index(inplace=True);
df

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront , Regent Park"
3,M6A,North York,"Lawrence Heights , Lawrence Manor"
4,M7A,Queen's Park,Not assigned
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge , Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens , Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson , Garden District"


Adding data to a dataframe
dropping unassigned boroughs (inplace so that the df will update)
grouping and joining neighborhoods with a comma

In [6]:
df['Neighbourhood'] = [df['Neighbourhood'][i].replace('Not assigned',df['Borough'][i]) for i in range(0,len(df['Neighbourhood']))]
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront , Regent Park"
3,M6A,North York,"Lawrence Heights , Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [7]:
df.shape[0]

103